In [1]:
from imports import *
import traces_rdf
%matplotlib inline

Welcome to JupyROOT 6.28/10


In [2]:
saveout = None
redir = None
def pipeout(filename=None):
    """Emulate '>>' redirection by changing sys.stdout to write to 'filename'.
       If 'filename' is empty, closes the output and restores stdout."""
    global saveout
    global redir
    if (filename is None and saveout):
        redir.close()
        sys.stdout = saveout
        return
    saveout = sys.stdout
    redir = open(filename, "w")
    sys.stdout = redir

In [3]:
# Dictionaries of information for the three detectors; 1 keV phonon-only (DMCNR bulk)
lite1 = {"File":'data/TES_scaling/TES-slowOnly-CDMSlite1-1keV_51240713_0000.root'}
lite2 = {"File":'data/TES_scaling/TES-slowOnly-CDMSlite2-1keV_51240713_0000.root'}
izip5 = {"File":'data/TES_scaling/TES-slowOnly-iZIP5-1keV_51240713_0000.root'}

lite1 = {"File":'data/TES_scaling/TES-slowCenter-CDMSlite1-1keV_51240713_0000.root'}
lite2 = {"File":'data/TES_scaling/TES-slowCenter-CDMSlite2-1keV_51240713_0000.root'}
izip5 = {"File":'data/TES_scaling/TES-slowCenter-iZIP5-1keV_51240713_0000.root'}

In [4]:
# Templates for each of the three Soudan runs
batsconf = os.environ["CDMSBATS_CONFIG"]
templates = os.path.join(batsconf,"PulseTemplates","files")

lite1["Template"] = {"File":os.path.join(templates,"Z14_Templates_R134.root"),
                     "Hist":"zip14_cdmsliteI_pos/PT"}

lite2["Template"] = {"File":os.path.join(templates,"Z4_Templates_R135.root"),
                     "Hist":"zip4_cdmsliteI_pos/PT"}

izip5["Template"] = {"File":os.path.join(templates,"Z4_Templates_R133.root"),
                     "Hist":"zip4_pos/PT"}

In [5]:
def getTemplate(templset):
    """Extract template bin values from TH1D in CDMSbats file."""
    troot = ROOT.TFile(templset["File"])
    thist = troot.Get(templset["Hist"])     # This is TH1D
    return np.asarray([thist.GetBinContent(i) for i in range(1,thist.GetNbinsX()+1)])

In [6]:
def getChannel(dataset, ch, event=0):
    """Extract single channel trace from dataset, with computed quantities."""
    ich = list(dataset["Channels"]).index(ch)    # getTES uses channel index, not name
    _,dataset[ch],dataset[f"I0_{ch}"],dataset[f"E_{ch}"] = traces_rdf.getTES(dataset["File"],event,ich)
    binmax = np.argmax(dataset[ch])
    dataset[f"Imax_{ch}"] = np.average(dataset[ch][binmax-20:binmax+20])
    dataset[f"IvsE_{ch}"] = dataset[f"Imax_{ch}"]/dataset[f"E_{ch}"]

In [7]:
def getTraceInfo(dataset, event=0):
    """Populate dictionary of channel traces for given data file."""
    dataset["Bins"] = traces_rdf.getBins(dataset["File"],"TES",event)
    dataset["Channels"] = traces_rdf.getChannels(dataset["File"],"TES",event)

    dataset["Psum"] = np.zeros_like(dataset["Bins"])
    dataset["Esum"] = 0.
    
    chans = dataset["Channels"]
    for ch in chans:
        getChannel(dataset, ch, event)
        dataset["Psum"] += dataset[ch]
        dataset["Esum"] += dataset[f"E_{ch}"]

    binmax = np.argmax(dataset["Psum"])
    dataset["Imax"] = np.average(dataset["Psum"][binmax-20:binmax+30])
    dataset["IvsE"] = dataset["Imax"]/dataset["Esum"]

    if "Template" in dataset:
        dataset["Ptemplate"] = getTemplate(dataset["Template"])

In [8]:
def reportIvsE(dataset):
    """Compute IvsE using peak of each channel trace in dataset. Traces have
       already been baseline-subtracted and flipped (via traces_rdf.getTES())."""
    for ch in dataset["Channels"]:
        print(ch," Imax",dataset[f"Imax_{ch}"],"E",dataset[f"E_{ch}"],
              " IvsE ",dataset[f"IvsE_{ch}"]*1e6," uA/MeV")
        
    print("Sum Imax",dataset["Imax"],"E",dataset["Esum"],
          " IvsE ",dataset["IvsE"]*1e6," uA/MeV")

In [9]:
def plotTraces(dataset,title):
    for ch in dataset["Channels"]:
        plt.plot(dataset["Bins"],dataset[ch],label=ch)
    plt.plot(dataset["Bins"],dataset["Psum"],label="Psum")
    plt.plot(dataset["Bins"],dataset["Ptemplate"]*dataset["Imax"],label="Template")
    plt.legend()
    plt.title(title)
    plt.xlabel("Time [us]")
    plt.ylabel("TES trace [uA]")

In [10]:
def IvsEatTime(dataset, time):
    """Compute IvsE by comparing trace value at a given time to the template."""
    tbin = np.where(dataset["Bins"]>=time)[0][0]
    IatT = np.average(dataset["Psum"][tbin-20:tbin+20])
    TmpAtT = dataset["Ptemplate"][tbin]
    ImaxAtT = IatT/TmpAtT
    IvsEatT = ImaxAtT/dataset["Esum"]
    print("Psum at",time,"us (bin",tbin,") I",IatT," Imax ",ImaxAtT," IvsE ",IvsEatT*1e6,"uA/MeV")

In [11]:
def getNtuple(dataset):
    """Pull g4dmcEvents information from input file to get sample averages."""
    from cats.cdataframe import CDataFrame
    branches = ["EventNum","DetNum","PhononE"]
    dataset["Events"] = CDataFrame("G4SimDir/g4dmcEvent", dataset["File"]).Filter("!Empty").AsNumpy(branches)

In [12]:
def fillEventIvsE(dataset):
    """Process dataset traces and copy summed IvsE value into Numpy event data."""
    if "Events" not in dataset: getNtuple(dataset)
    if "IvsE" in dataset["Events"]: return     # Already done for this sample

    dataset["Events"]["Imax"] = np.zeros_like(dataset["Events"]["EventNum"])
    dataset["Events"]["IvsE"] = np.zeros_like(dataset["Events"]["EventNum"])
    for ev in dataset["Events"]["EventNum"]:
        select = np.where(dataset["Events"]["EventNum"]==ev)
        getTraceInfo(dataset, event=ev)
        dataset["Events"]["Imax"][select] = dataset["Imax"]
        dataset["Events"]["IvsE"][select] = dataset["IvsE"]

In [13]:
fillEventIvsE(lite1)
print(np.average(lite1["Events"]["IvsE"])*1e6," +- ",np.std(lite1["Events"]["IvsE"])*1e6,"uA/MeV")

50.517536750638776  +-  0.8133392724276263 uA/MeV


In [14]:
fillEventIvsE(lite2)
print(np.average(lite2["Events"]["IvsE"])*1e6," +- ",np.std(lite2["Events"]["IvsE"])*1e6,"uA/MeV")

50.517536750638776  +-  0.8133392724276263 uA/MeV


In [ ]:
fillEventIvsE(izip5)
print(np.average(izip5["Events"]["IvsE"])*1e6," +- ",np.std(izip5["Events"]["IvsE"])*1e6,"uA/MeV")

In [ ]:
traces_rdf.verbose=False
getTraceInfo(lite1)
getTraceInfo(lite2)
getTraceInfo(izip5)

In [ ]:
pipeout("IvsE_CDMSlite1.txt")
reportIvsE(lite1)
IvsEatTime(lite1,2000.)
pipeout()

plotTraces(lite1, "CDMSlite1 1 keV phonon-only at center")
plt.savefig("traces_CDMSlite1.png")

In [ ]:
pipeout("IvsE_CDMSlite2.txt")
reportIvsE(lite2)
IvsEatTime(lite2,2000.)
pipeout()

plotTraces(lite2, "CDMSlite2 1 keV phonon-only at center")
plt.savefig("traces_CDMSlite2.png")

In [ ]:
pipeout("IvsE_iZIP5.txt")
reportIvsE(izip5)
IvsEatTime(izip5,2000.)
pipeout()

plotTraces(izip5,"iZIP5 1 keV phonon-only at center")
plt.savefig("traces_iZIP5.png")